In [1]:
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

rng = np.random.default_rng(12345)

In [2]:
def GBM(S0, T, mu, sigma, Npaths):

    # The number of trading days based on 252 trading days per year
    Nsteps = int(252 * T) 
    
    t, dt = np.linspace(0, T, Nsteps+1, retstep=True)
    S = np.zeros((Nsteps+1, Npaths))
    S[0, :] = S0
    
    for n in range(Nsteps):
        dW = np.sqrt(dt) * rng.normal(0, 1, Npaths)
        S[n+1, :] = S[n, :] * (1 + mu * dt + sigma * dW[:])
        
    return t, S

### Asian Option

Asian options are a category of financial derivatives whose payoff is contingent on the average market price of the underlying asset over a specific timeframe.

- Asian Call Option: $(\bar{S} - K)^+$, where $K$ stands for the strike price.
- Asian Put option: $(K - \bar{S})^+$.

Average Price Calculation:
The average price $\bar{S}$ of the underlying asset is calculated by using the formula:
$$ \bar{S} = \frac{1}{n} \sum_{i=1} ^n S(t_i)$$
where $S(t_i)$ represents the price of the underlying asset at specific monitoring dates $t_1, t_2, \dots, t_n$, and $n$ is the total number of these dates.

In [3]:
def AsianOption(S0, K, T, r, sigma, Npaths, option_type='call'):
    
    if option_type not in ['call', 'put']:
        raise ValueError("Error: Invalid option type. Please choose 'call' or 'put'.")
    
    t, S = GBM(S0=S0, T=T, mu=r, sigma=sigma, Npaths=Npaths)
    
    # Calculate payoff based on option type
    if option_type == 'call':
        fST = np.exp(-r * T) * np.maximum(np.mean(S, axis=0) - K, 0)
    elif option_type == 'put':
        fST = np.exp(-r * T) * np.maximum(K - np.mean(S, axis=0), 0)
        
    price = np.mean(fST)
    var = np.var(fST)
    
    return price, var

In [4]:
# parameters
S0 = 110
K = 100
T = 1
r = 0.05
sigma = 0.01
Npaths = 10**3

AsianPrice, AsianVar = AsianOption(S0, K, T, r, sigma, Npaths)
AsianSEM = np.sqrt(AsianVar / Npaths)

# Display the results
print("The Asian option Price: ", round(AsianPrice, 4), "+/-", format(1.96*AsianSEM, '.2g'))

The Asian option Price:  12.1994 +/- 0.038


### Geometric Average Option

Replacing the arithmetic average $\bar{S}$ in Asian option with
$$ (\prod_{i=1} ^n S(t_i))^{\frac{1}{n}}$$
produces an option on the geometric average of the underlying asset price. Such options are seldom if ever found in practice, but they are useful as test cases for computiational procedures and as a basis for approximating ordinary Asian options.

To avoid overflow, I am going to use another formula to use logarithm:
$$ (\prod_{i=1} ^n S(t_i))^{\frac{1}{n}} = \exp(\frac{1}{n} \sum_{i=1} ^n \log(S(t_i)))$$

In [5]:
def GeoAvgOption(S0, K, T, r, sigma, Npaths, option_type='call'):
    
    if option_type not in ['call', 'put']:
        raise ValueError("Error: Invalid option type. Please choose 'call' or 'put'.")
    
    t, S = GBM(S0=S0, T=T, mu=r, sigma=sigma, Npaths=Npaths)
    
    # Calculate payoff based on option type
    if option_type == 'call':
        fST = np.exp(-r * T) * np.maximum(np.exp(np.log(S).mean(axis=0)) - K, 0)
    elif option_type == 'put':
        fST = np.exp(-r * T) * np.maximum(K - np.exp(np.log(S).mean(axis=0)), 0)
        
    price = np.mean(fST)
    var = np.var(fST)
    
    return price, var

In [6]:
# parameters
S0 = 110
K = 100
T = 1
r = 0.05
sigma = 0.01
Npaths = 10**3

GeoAvgPrice, GeoAvgVar = GeoAvgOption(S0, K, T, r, sigma, Npaths)
GeoAvgSEM = np.sqrt(GeoAvgVar / Npaths)

# Display the results
print("The Geometric Average option Price: ", round(GeoAvgPrice, 4), "+/-", format(1.96*GeoAvgSEM, '.2g'))

The Geometric Average option Price:  12.1811 +/- 0.039


### Barrier Option

Barrier options are exotic options with payoffs that depend on the underlying asset's price reaching a specified barrier level during the option's lifespan.

1. Down-and-Out: This option becomes invalid (knocked out) if the asset's price drops below the barrier level ($S_b$) at any point.
$$
\begin{cases}
\max(S_T-K, 0), & {\rm if} \min_{t_i} S_{t_i} > S_b \\
0, & {\rm otherwise}
\end{cases}
$$


2. Down-and-In: Contrary to the down-and-out, a down-and-in option becomes active (gets "knocked in") only if the asset's price drops below the barrier level at least once during the option's lifetime.
$$
\begin{cases}
\max(S_T-K, 0), & {\rm if} \min_{t_i} S_{t_i} \leq S_b \\
0, & {\rm otherwise}
\end{cases}
$$


3. Up-and-Out: This option becomes invalid (knocked out) if the asset's price rises above the barrier level ($S_b$) at any point.
$$
\begin{cases}
\max(S_T-K, 0), & {\rm if} \max_{t_i} S_{t_i} < S_b \\
0, & {\rm otherwise}
\end{cases}
$$


4. Up-and-In: Contrary to the Up-and-Out, this option becomes active (knocked in) only if the asset's price rises above the barrier level at least once during the option's lifetime.
$$
\begin{cases}
\max(S_T-K, 0), & {\rm if} \max_{t_i} S_{t_i} \geq S_b \\
0, & {\rm otherwise}
\end{cases}
$$

In [7]:
def BarrierOption(S0, Sb, K, T, r, sigma, Npaths, option_type='call', barrier_direction='up', activation_type='out'):
    
    # Validate option_type, barrier_direction and activation_type
    if option_type not in ['call', 'put']:
        raise ValueError("Invalid option type. Please choose 'call' or 'put'.")
    if barrier_direction not in ['up', 'down']:
        raise ValueError("Invalid barrier direction. Please choose 'up' or 'down'.")
    if activation_type not in ['in', 'out']:
        raise ValueError("Invalid activation type. Please choose 'in' or 'out'.")

    t, S = GBM(S0=S0, T=T, mu=r, sigma=sigma, Npaths=Npaths)
    
    # Initialise indicator based on barrier_direction and activation_type

    if barrier_direction == 'up':
        maxS = np.amax(S, axis=0)
        if activation_type == 'out':
            indicator = np.heaviside(Sb - maxS, 0.5)
        elif activation_type == 'in':
            indicator = np.heaviside(maxS - Sb, 0.5)
            
    elif barrier_direction == 'down':
        minS = np.amin(S, axis=0)
        if activation_type == 'out':
            indicator = np.heaviside(minS - Sb, 0.5)
        elif activation_type == 'in':
            indicator = np.heaviside(Sb - minS, 0.5)

    # Calculate payoff based on option type
    if option_type == 'call':
        payoff = np.maximum(S[-1, :] - K, 0)
    elif option_type == 'put':
        payoff = np.maximum(K - S[-1, :], 0)
        
    fST = np.exp(-r * T) * payoff * indicator
    
    price = np.mean(fST)
    var = np.var(fST)
    
    return price, var


In [8]:
# parameters
S0 = 110
K = 100
T = 1
r = 0.05
sigma = 0.01
Npaths = 10**3

#Barrier
Sb = 80

#Down-and-out
Price_dn_out, Var_dn_out = BarrierOption(S0, Sb, K, T, r, sigma, Npaths, 'call', 'down', 'out')
SEM_dn_out = np.sqrt(Var_dn_out / Npaths)

# Display the results
print("The Barrier(Down-and-out) option Price: ", round(Price_dn_out, 4), "+/-", format(1.96*SEM_dn_out, '.2g'))

The Barrier(Down-and-out) option Price:  14.9347 +/- 0.07


### LookBack Option

Like Barrier Options, lookback options depends on external values of the underlying asset price. Lookback calls expiring at $T = t_n$ have payoffs
$$ (S(T) - \min_{i=1, \dots, n} S(t_i) ) $$

In [9]:
def LookbackOption(S0, K, T, r, sigma, Npaths, option_type='call'):
    # Validate option_type, barrier_direction and activation_type
    if option_type not in ['call', 'put']:
        raise ValueError("Invalid option type. Please choose 'call' or 'put'.")

    t, S = GBM(S0=S0, T=T, mu=r, sigma=sigma, Npaths=Npaths)
    
    if option_type == 'call':
        minS = np.amin(S, axis=0)
        fST = np.exp(-r * T) * np.maximum(S[-1, :] - minS, 0)
    elif option_type == 'put':
        maxS = np.amax(S, axis=0) 
        fST = np.exp(-r * T) * np.maximum(maxS - S[-1, :] , 0)
    
    price = np.mean(fST)
    var = np.var(fST)
    
    return price, var

In [10]:
# parameters
S0 = 110
K = 100
T = 1
r = 0.05
sigma = 0.01
Npaths = 10**3

lbPrice, lbVar = LookbackOption(S0, K, T, r, sigma, Npaths)
lbSEM = np.sqrt(lbVar / Npaths)

# Display the results
print("The Lookback option Price: ", round(lbPrice, 4), "+/-", format(1.96*lbSEM, '.2g'))

The Lookback option Price:  5.4222 +/- 0.065
